# Estimando distribuciones (parte 1)

**Estudiante**: Germán Otero
**Número estudiante**: 138796

El objetivo de esta notebook es explorar una primera manera de aproximar $p(y|x)$ y $p(x|y)$ en un set de datos tabular. En este set de datos $x$ tiene valores discretos, $x\in\mathbb{D}^k$, y el target $y$ es un booleano, $y\in\{0,1\}$.

## Imports

In [2]:
import numpy as np
import pandas as pd
import random

## Cargamos los datos

In [3]:
df = pd.read_csv('./tennis.csv', delimiter=',', header=0)
df

,Day,Outlook,Temp,Humidity,Wind,Tennis
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


### Eliminamos la columna Day 

In [4]:
df = df.drop('Day', axis=1)
df

,Outlook,Temp,Humidity,Wind,Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [5]:
X_names = df.columns.to_list()[:-1]
X_names

['Outlook', 'Temp', 'Humidity', 'Wind']

Guardamos en la variable $X$ todas las features del dataset.

In [6]:
X = df.iloc[:,0:-1]
X

,Outlook,Temp,Humidity,Wind
0,Sunny,Hot,High,Weak
1,Sunny,Hot,High,Strong
2,Overcast,Hot,High,Weak
3,Rain,Mild,High,Weak
4,Rain,Cool,Normal,Weak
5,Rain,Cool,Normal,Strong
6,Overcast,Cool,Normal,Strong
7,Sunny,Mild,High,Weak
8,Sunny,Cool,Normal,Weak
9,Rain,Mild,Normal,Weak


In [7]:
Y_name = df.columns.to_list()[-1]
Y_name

'Tennis'

Guardamos en $Y$ el objetivo

In [8]:
Y = df.iloc[:,-1]
Y

0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: Tennis, dtype: object

## Construimos una tabla de observaciones

En este paso vamos a crear una tabla de observaciones. Esta tabla tiene que contener la frecuencia de cada observación. Para este ejemplo tomaremos a $x$ como **Outlook**.

Calcule las dimensiones de la tabla

In [9]:
# Cantidad total de elementos
N = df.shape[0]

# Elementos únicos de la clase Outlook
xvalues = df.iloc[:,0].unique() # Sabemos que la columna 0 es la clase Outlook
dimx = len(xvalues)

# Elementos únicos del objetivo
yvalues = df.iloc[:,-1].unique()
dimy = len(yvalues)

print(f'Cantidad total de elementos: {N}')
print(f'Elementos únicos de la clase Outlook: {xvalues}')
print(f'Elementos únicos del objetivo: {yvalues}')

Cantidad total de elementos: 14
Elementos únicos de la clase Outlook: ['Sunny' 'Overcast' 'Rain']
Elementos únicos del objetivo: ['No' 'Yes']


Calculamos la tabla de frecuencia.

In [10]:
obs = pd.DataFrame(0, columns=yvalues, index=xvalues)


## Llene la tabla de observaciones

for i in range(len(df)):
    # df.iloc[i,0] es el valor de la clase Outlook
    # df.iloc[i,-1] es el valor del objetivo
    obs.loc[df.iloc[i,0], df.iloc[i,-1]] += 1

obs

,No,Yes
Sunny,3,2
Overcast,0,4
Rain,2,3


## Aproximación de la distribución conjunta $p(x,y)$

Tome a $x$ como Outlook y aproxime la distribución conjunta utilizando la tabla de observaciones. 

In [11]:
joint_x_y = obs.values / N
joint_x_y = pd.DataFrame(joint_x_y, columns=yvalues, index=xvalues)
joint_x_y

,No,Yes
Sunny,0.214286,0.142857
Overcast,0.000000,0.285714
Rain,0.142857,0.214286


1. ¿Qué significa el valor calculado en los índices "Sunny", "Yes"?

2. ¿Justifique el resultado de la pareja "Overcast", "No"?

**[Nota de estudiante]**
1. La probabilidad de que se juegue al tenis y que esté soleado.
2. Dentro de las observaciones, no se ha dado el caso de que esté nublado y se haya jugado al tenis.

## Aproximamos $p(y|x)$

Tome a $x$ como **Outlook** y estime la probabilidad condicional de $y$ dado $x$. Luego realice una muestra de 10 valores de $y$ dado $x = Sunny$.

Calculamos la cantidad de entradas por cada valor distinto de $x$.

In [12]:
m = obs.sum(axis=1)
obs['m'] = m
m

Sunny       5
Overcast    4
Rain        5
dtype: int64

Calculamos la cantidad de entradas por cada valor distinto de $y$.

In [13]:
l = obs.sum(axis=0)
obs.loc['l'] = l
l

No      5
Yes     9
m      14
dtype: int64

In [14]:
obs

,No,Yes,m
Sunny,3,2,5
Overcast,0,4,4
Rain,2,3,5
l,5,9,14


Calcule la probabilidad condicional de $y$ dado $x$.

In [14]:
p_y_x = pd.DataFrame(0, columns=yvalues, index=xvalues)

## Llene la tabla de probabilidades condicionales p(y|x)
for i in range(dimx):
    for j in range(dimy):
        if m[i] != 0:
            p_y_x.iloc[i,j] = obs.iloc[i,j] / m[i]

p_y_x

/var/folders/1s/blr559y52fn_mdjqmdysyrk80000gn/T/ipykernel_30546/4245119612.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if m[i] != 0:
/var/folders/1s/blr559y52fn_mdjqmdysyrk80000gn/T/ipykernel_30546/4245119612.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_y_x.iloc[i,j] = obs.iloc[i,j] / m[i]
/var/folders/1s/blr559y52fn_mdjqmdysyrk80000gn/T/ipykernel_30546/4245119612.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  p_y_x.iloc

,No,Yes
Sunny,0.6,0.4
Overcast,0.0,1.0
Rain,0.4,0.6


3. ¿La suma de cada fila siempre tiene que dar 1? ¿Por qué?

4. ¿Y si la suma de las columnas?

**[nota del estudiante]**

3. Se debe a que los eventos son complementarios, es decir, p(x='Sunny'|y='No') y p(x='Sunny'|y='Yes') son mutuamente exclusivos y son todos los casos posibles para x='Sunny'. Otra forma de ver esto es que todos los valores observados en las filas, constituyen una distribución en sí misma por lo que -por definición- la suma de todos los valores debe ser igual a uno.

4. En el caso de las columnas, no se trata de una distribución por lo que no podemos predecir qué valores o resultados deberían dar.

Realice 10 muestras de $y$ dado $x = Sunny$.

Puede utilizar la función random.choice de numpy. https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html

In [15]:
sampled_values = np.random.choice(yvalues, size=10, p=p_y_x.loc['Sunny'].values)

sampled_values

array(['Yes', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes'],
      dtype=object)

5. ¿Qué pasaría si utilizamos $x = Overcast$ en vez de $x = Sunny$? ¿Tiene sentido que pase esto? ¿Por qué?

**[nota del estudiante]**

5. De acuerdo a las observaciones, si el outlook es 'Overcast', siempre se jugó al tenis.

## Aproximamos $p(x|y)$
Tome a $x$ como Outlook y estime la probabilidad condicional de $x$ dado $y$ basandose en la tabla de observaciones. Luego realice 10 muestras de $x$ dado $y = Yes$.

$p(x|y)$

In [17]:
p_x_y = pd.DataFrame(0, columns=yvalues, index=xvalues)

## Llene la tabla de probabilidades condicionales p(x|y)
for j in range(dimy):
    for i in range(dimx):
        if l[j] != 0:
            p_x_y.iloc[i,j] = obs.iloc[i,j] / l[j]

p_x_y

/var/folders/1s/blr559y52fn_mdjqmdysyrk80000gn/T/ipykernel_1789/4009333379.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if l[j] != 0:
/var/folders/1s/blr559y52fn_mdjqmdysyrk80000gn/T/ipykernel_1789/4009333379.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_x_y.iloc[i,j] = obs.iloc[i,j] / l[j]
/var/folders/1s/blr559y52fn_mdjqmdysyrk80000gn/T/ipykernel_1789/4009333379.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  p_x_y.iloc[i,

,No,Yes
Sunny,0.6,0.222222
Overcast,0.0,0.444444
Rain,0.4,0.333333


Muestreo

In [15]:
obs

,No,Yes,m
Sunny,3,2,5
Overcast,0,4,4
Rain,2,3,5
l,5,9,14


In [18]:
sampled_values = np.random.choice(xvalues, size=10, p=p_x_y.loc[:, 'Yes'].values)

sampled_values

array(['Sunny', 'Overcast', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny',
       'Overcast', 'Overcast', 'Overcast'], dtype=object)

## Aproxime $p(y,o,h,w,t)$

Aproxime la proabilidad conjunta del Tennis (y), Outlook (o), Humidity (h), Wind (w), Temp (t).

Recuerde que $p(y,o,h,w,t)$ = $p(y)$.$p(o|y)$.$p(h|y,o)$.$p(w|y,o,h)$.$p(t|y,o,h,w)$

Calcule P(y)

In [19]:
# P(Y)

p_y = l / N
p_y

No     0.357143
Yes    0.642857
m      1.000000
dtype: float64

Calcule P(o|y)

In [26]:
p_o_y = pd.DataFrame(0, columns=yvalues, index=xvalues)

for j in range(dimy):
    for i in range(dimx):
        if l[j] != 0:
            p_o_y.iloc[i,j] = obs.iloc[i,j] / l[j]
p_o_y

/var/folders/1s/blr559y52fn_mdjqmdysyrk80000gn/T/ipykernel_1789/3664626202.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if l[j] != 0:
/var/folders/1s/blr559y52fn_mdjqmdysyrk80000gn/T/ipykernel_1789/3664626202.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_o_y.iloc[i,j] = obs.iloc[i,j] / l[j]
/var/folders/1s/blr559y52fn_mdjqmdysyrk80000gn/T/ipykernel_1789/3664626202.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  p_o_y.iloc[i,

,No,Yes
Sunny,0.6,0.222222
Overcast,0.0,0.444444
Rain,0.4,0.333333


Calcule P(h|y,o)

Recomendamos usar la función *crosstab* de pandas. En este link pueden encontrar un ejemplo de su uso: https://www.geeksforgeeks.org/pandas-crosstab-function-in-python/

In [ ]:
# Calcule la tabla de frecuencia
# p(h|y,o) = 
p_h_yo = 

# Luego se divide cada fila por la suma de sus elementos, puede usar las funciones div y sum de pandas
p_h_yo = ...

# No se oliden de llenar los valores NaN!!!

# Cambiamos los nombres de los indices (si es que lo precisa)


p_h_yo

Calcule P(w|y,o,h)

In [ ]:
p_w_yoh = ...
p_w_yoh = ...
p_w_yoh

Calcule P(t|y,o,h,w)

In [ ]:
p_t_yohw = ...
p_t_yohw = ...
p_t_yohw

Calcule P(y,o,h,w,t)

In [ ]:
# Definimos una función que nos calcula la probabilidad conjunta usando la regla del producto.
def calculate_prob(y,o,h,w,t):
    "Calculate the probability of occurrence of a row"
    ...

In [ ]:
prob = calculate_prob('Yes','Sunny','Normal','Weak','Cool')

print(f'P(Yes|Sunny,Normal,Weak,Cool) = {prob}')
print(f"Cantidad de observaciones: {prob*N}")

Definimos el muestreo de datos completos.

Primero generamos un *y* utilizando *p(y)* y luego seguimos con las probabilidades condicionales.

In [ ]:
def sample_from_y(y):
    "Generates a sample of weather conditions based on a specific y"
    ...

def sample():
    "Generates a sample of weather conditions based on a random y"
    ...

In [ ]:
samples = np.array([sample() for _ in range(len(df))])
new_df = pd.DataFrame(samples, columns=['Tennis', 'Outlook', 'Humidity', 'Wind', 'Temp'])
new_df